In [1]:
import torch

can create a tensor full of ones with `tensor.ones`

In [15]:
a = torch.ones(3)
a

tensor([1., 1., 1.])

can give multiple dimennsions, doesn't need to be a tuple

In [16]:
b = torch.ones(3,3)
b

tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]])

You can use `dtype` to get the type (`float32` by default) and `shape` to get the shape

In [17]:
b.dtype, b.shape

(torch.float32, torch.Size([3, 3]))

you can use index by multiple axes, just like numpy

In [18]:
b[1,2] = 99.1
b

tensor([[ 1.0000,  1.0000,  1.0000],
        [ 1.0000,  1.0000, 99.1000],
        [ 1.0000,  1.0000,  1.0000]])

and you can slice on them the same way

In [19]:
b[1:, 1:]

tensor([[ 1.0000, 99.1000],
        [ 1.0000,  1.0000]])

Imagine we have an RGB image (here represented by a 5x5 array of RGB triples) and we want to convert it to grayscale. We might weight the conversion as such:

In [21]:
img_t = torch.randn(3, 5, 5) # shape [channels, rows, columns]
weights = torch.tensor([0.2126, 0.7152, 0.0722])

let's set a batch size of 2

In [22]:
batch_t = torch.randn(2, 3, 5, 5)

so our RGB channels are now in column zero in img_t and column one in batch_t - but they are always third from last so we can use -3 to refer to them. Here's the grayscale conversion without using our weights:

In [23]:
img_gray_naive = img_t.mean(-3)
batch_gray_naive = batch_t.mean(-3)
img_gray_naive.shape, batch_gray_naive.shape

(torch.Size([5, 5]), torch.Size([2, 5, 5]))

now we can try it with the weights. First, we can expand the weights size to match the image size.
- It's not clear to me why the author used -1 instead of 1; the results are identical
- unsqueeze_ modifies the tensor in place, while unsqueeze returns a new view on the same data but doesn't modify the shape of the underlying tensor

In [24]:
unsqueezed_weights = weights.unsqueeze(-1).unsqueeze_(-1)
unsqueezed_weights, unsqueezed_weights.shape

(tensor([[[0.2126]],
 
         [[0.7152]],
 
         [[0.0722]]]),
 torch.Size([3, 1, 1]))

Torch has a function `einsum`, adapted from NumPy, which specifies an indexing [mini-language](https://rockt.github.io/2018/04/30/einsum). Broadcasting is done using three dots

In [27]:
img_gray_weighted_ein = torch.einsum('...chw,c->...hw', img_t, weights)
batch_gray_weighted_ein = torch.einsum('...chw,c->...hw', batch_t, weights)
img_gray_weighted_ein.shape, batch_gray_weighted_ein.shape

(torch.Size([5, 5]), torch.Size([2, 5, 5]))

I don't understand what's going on there at all, and [it has been noticed by others](http://nlp.seas.harvard.edu/NamedTensor).

To try and improve the situation, torch added _named tensors_.

[tutorial](https://pytorch.org/tutorials/intermediate/named_tensor_tutorial.html), [docs](https://pytorch.org/docs/stable/named_tensor.html)

In [28]:
weights_named = torch.tensor([0.2126, 0.7152, 0.0722], names=['channels'])
weights_named

/var/folders/cj/3fpctnd52vv37y9gpdgg7c200000gn/T/ipykernel_61057/2371314847.py:1: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/c10/core/TensorImpl.h:1761.)
  weights_named = torch.tensor([0.2126, 0.7152, 0.0722], names=['channels'])


tensor([0.2126, 0.7152, 0.0722], names=('channels',))

- When we already have a tensor and want to add names, we can use `refine_names`
- `...` allows you to elide any number of dimensions so you can name the ones you want
- `rename` allows you to overwrite or drop names

In [31]:
img_named = img_t.refine_names('channels', 'rows', 'columns')
batch_named = batch_t.refine_names(..., 'channels', 'rows', 'columns')
img_named.names, batch_named.names

(('channels', 'rows', 'columns'), (None, 'channels', 'rows', 'columns'))

In addition to dimension checks, torch will now check names when we do operations.

It does not automatically align dimensions, so we need to do so explicitly.

In [33]:
weights_aligned = weights_named.align_as(img_named)
weights_aligned, weights_named

(tensor([[[0.2126]],
 
         [[0.7152]],
 
         [[0.0722]]], names=('channels', 'rows', 'columns')),
 tensor([0.2126, 0.7152, 0.0722], names=('channels',)))

Functions accepting dimensions, like sum, accept names instead:

In [34]:
gray_named = (img_named * weights_aligned).sum('channels')
gray_named.shape, gray_named.names

(torch.Size([5, 5]), ('rows', 'columns'))

combining dimensions with different names yields an error

In [35]:
gray_named = (img_named[..., :3] * weights_named).sum('channels')

RuntimeError: Error when attempting to broadcast dims ['channels', 'rows', 'columns'] and dims ['channels']: dim 'columns' and dim 'channels' are at the same position from the right but do not match.

If we want to use named tensors with functions that don't handle named tensors, drop the names by renaming them to `None`

In [38]:
gray_named.rename(None)

tensor([[-0.5496,  1.9017,  0.1883,  0.6561, -0.6928],
        [-1.0958,  0.1077,  1.0926,  1.0960,  0.0304],
        [-0.1555, -1.2855,  0.9975,  0.7204,  0.3737],
        [ 0.8061,  1.2827,  1.5501, -1.0633,  1.0668],
        [-0.0820,  0.5158,  0.4953,  1.0490,  0.2519]])

The remainder of this book will use the unnamed version of tensors because this is still experimental